In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather

In [2]:
import glob
import os

In [3]:
#path for 11 models
path = "//home/mlopez/EXEC/Precip annual 11 models/rcp45_prcptot_annual/"

In [4]:
files = list(glob.glob(os.path.join(path,'*.*')))

In [5]:
files

['//home/mlopez/EXEC/Precip annual 11 models/rcp45_prcptot_annual/BNU-ESM_rcp45_prcptot_annual.nc',
 '//home/mlopez/EXEC/Precip annual 11 models/rcp45_prcptot_annual/INM-CM4_rcp45_prcptot_annual.nc',
 '//home/mlopez/EXEC/Precip annual 11 models/rcp45_prcptot_annual/CMCC-CMS_rcp45_prcptot_annual.nc',
 '//home/mlopez/EXEC/Precip annual 11 models/rcp45_prcptot_annual/IPSL-CM5A-LR_rcp45_prcptot_annual.nc',
 '//home/mlopez/EXEC/Precip annual 11 models/rcp45_prcptot_annual/IPSL-CM5B-LR_rcp45_prcptot_annual.nc',
 '//home/mlopez/EXEC/Precip annual 11 models/rcp45_prcptot_annual/NorESM1-M_rcp45_prcptot_annual.nc',
 '//home/mlopez/EXEC/Precip annual 11 models/rcp45_prcptot_annual/CanESM2_rcp45_prcptot_annual.nc',
 '//home/mlopez/EXEC/Precip annual 11 models/rcp45_prcptot_annual/GFDL-ESM2M_rcp45_prcptot_annual.nc',
 '//home/mlopez/EXEC/Precip annual 11 models/rcp45_prcptot_annual/HadGEM2-CC_rcp45_prcptot_annual.nc',
 '//home/mlopez/EXEC/Precip annual 11 models/rcp45_prcptot_annual/MPI-ESM-LR_rcp4

# Create DF of all models

In [6]:
dflist = []

In [9]:
#Takes some seconds
for file in files:
    df = pcdf.load_as_df(file)
    dflist.append(df)

/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()
/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()
/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between

In [10]:
dflist

[                                     prcptot
 lat       lon        time                   
 66.623306 -89.045212 1950-01-01   220.538116
                      1951-01-01   208.522812
                      1952-01-01   240.642914
                      1953-01-01   194.157272
                      1954-01-01   193.584473
 ...                                      ...
 40.041039 -74.045807 2096-01-01  1179.984375
                      2097-01-01  1105.808350
                      2098-01-01  1104.722290
                      2099-01-01  1407.243530
                      2100-01-01  1300.690063
 
 [11507106 rows x 1 columns],                                      prcptot
 lat       lon        time                   
 66.623306 -89.045212 1950-01-01   251.282410
                      1951-01-01   154.389954
                      1952-01-01   251.749191
                      1953-01-01   205.387619
                      1954-01-01   187.220108
 ...                                      ...
 40

In [11]:
#Takes 3 min
%time df11 = pd.concat(dflist)

CPU times: user 2min 49s, sys: 6min 56s, total: 9min 46s
Wall time: 1min 14s


In [12]:
df11

prcptot
lat       lon        time                   
66.623306 -89.045212 1950-01-01   220.538116
                     1951-01-01   208.522812
                     1952-01-01   240.642914
                     1953-01-01   194.157272
                     1954-01-01   193.584473
...                                      ...
40.041039 -74.045807 2096-01-01  1475.714111
                     2097-01-01  1401.659790
                     2098-01-01  1467.047241
                     2099-01-01  1489.424561
                     2100-01-01  1331.375366

[126578166 rows x 1 columns]

In [13]:
dftgm = df11.reset_index()

In [14]:
dftgm

,lat,lon,time,prcptot
0,66.623306,-89.045212,1950-01-01,220.538116
1,66.623306,-89.045212,1951-01-01,208.522812
2,66.623306,-89.045212,1952-01-01,240.642914
3,66.623306,-89.045212,1953-01-01,194.157272
4,66.623306,-89.045212,1954-01-01,193.584473
...,...,...,...,...
126578161,40.041039,-74.045807,2096-01-01,1475.714111
126578162,40.041039,-74.045807,2097-01-01,1401.659790
126578163,40.041039,-74.045807,2098-01-01,1467.047241
126578164,40.041039,-74.045807,2099-01-01,1489.424561


# Get the mean for 11 models

In [15]:
#Takes 1 min 10s
%time dftgm2 = dftgm.groupby(["lat", "lon", "time"]).mean()

CPU times: user 1min 10s, sys: 3min 21s, total: 4min 31s
Wall time: 1min 28s


In [16]:
dftgm2

prcptot
lat       lon        time                   
40.041039 -89.045212 1950-01-01  1070.059814
                     1951-01-01  1088.270508
                     1952-01-01  1045.444214
                     1953-01-01   966.804138
                     1954-01-01  1044.817871
...                                      ...
66.623306 -61.296322 2096-01-01   575.799683
                     2097-01-01   535.829102
                     2098-01-01   551.541687
                     2099-01-01   558.133606
                     2100-01-01   566.340637

[11507106 rows x 1 columns]

# Define 3 periods of time: 

In [18]:
year_groups = {y:0 for y in range(1980,2011)}
year_groups.update({y:1 for y in range(2041,2071)})
year_groups.update({y:2 for y in range(2071,2101)})
#Get the mean of every period
dfC2 = dftgm2.reset_index()
dfp = dfC2.groupby([dfC2.time.dt.year.map(year_groups), "lat","lon"]).mean()

In [19]:
dfp.reset_index()

,time,lat,lon,prcptot
0,0.0,40.041039,-89.045212,998.437805
1,0.0,40.041039,-88.961884,1006.917603
2,0.0,40.041039,-88.878548,999.683350
3,0.0,40.041039,-88.795219,994.054321
4,0.0,40.041039,-88.711891,996.034241
...,...,...,...,...
228613,2.0,66.623306,-61.629639,572.109009
228614,2.0,66.623306,-61.546310,583.565735
228615,2.0,66.623306,-61.462978,576.401672
228616,2.0,66.623306,-61.379650,571.699158


# Historic Period

In [20]:
df_h = dfp.query("time==0.0")
df_h.reset_index()

,time,lat,lon,prcptot
0,0.0,40.041039,-89.045212,998.437805
1,0.0,40.041039,-88.961884,1006.917603
2,0.0,40.041039,-88.878548,999.683350
3,0.0,40.041039,-88.795219,994.054321
4,0.0,40.041039,-88.711891,996.034241
...,...,...,...,...
76201,0.0,66.623306,-61.629639,502.487579
76202,0.0,66.623306,-61.546310,511.931732
76203,0.0,66.623306,-61.462978,505.752686
76204,0.0,66.623306,-61.379650,501.764130


## Import dataframe with polygons

In [21]:
dfpolyshape = from_geofeather('Grid-TerritoiresGuides.feather')

/home/mlopez/EXEC/anaconda3/envs/python37-2/lib/python3.8/site-packages/geofeather/core.py:85: UserWarning: Grid-TerritoiresGuides.feather.crs coordinate reference system file is missing. No crs will be set for this GeoDataFrame.
  warnings.warn(


In [22]:
dfpolyshape

,lat,lon,TER_GUIDE,geometry
0,52.957191,-67.712730,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
1,52.957191,-67.629402,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."
2,52.957191,-67.546066,6opqr,"POLYGON ((-67.56730 52.91553, -67.57974 52.915..."
3,52.873859,-67.796059,6opqr,"POLYGON ((-67.75439 52.89329, -67.75439 52.832..."
4,52.873859,-67.712730,6opqr,"POLYGON ((-67.75439 52.83219, -67.75439 52.893..."
...,...,...,...,...
33193,45.041668,-72.208336,2c,"POLYGON ((-72.25000 45.00450, -72.25000 45.083..."
33194,45.041668,-72.125000,2c,"POLYGON ((-72.16666 45.00508, -72.16666 45.083..."
33195,45.041668,-72.041672,2c,"POLYGON ((-72.08334 45.00564, -72.08334 45.083..."
33196,45.041668,-71.958336,2c,"POLYGON ((-72.00000 45.00667, -72.00000 45.083..."


## Merge data with mask 

In [26]:
dfpr11TG = pd.merge(df_h, dfpolyshape, on=["lat","lon"])

In [27]:
dfpr11TG

,lat,lon,prcptot,TER_GUIDE,geometry
0,44.957512,-74.629120,970.615723,1a,"POLYGON ((-74.62590 44.99918, -74.58746 44.999..."
1,44.957512,-74.545792,969.900818,1a,"POLYGON ((-74.58746 44.99876, -74.58746 44.999..."
2,44.957512,-74.462463,971.484131,1a,"POLYGON ((-74.50413 44.99784, -74.50413 44.999..."
3,44.957512,-74.379128,975.254822,1a,"POLYGON ((-74.42079 44.99530, -74.42079 44.999..."
4,44.957512,-74.295799,983.041443,1a,"POLYGON ((-74.33746 44.99215, -74.33746 44.999..."
...,...,...,...,...,...
16563,52.873859,-67.629402,882.958679,6opqr,"POLYGON ((-67.67107 52.83219, -67.67107 52.915..."
16564,52.873859,-67.546066,880.971863,6opqr,"POLYGON ((-67.58773 52.83219, -67.58773 52.904..."
16565,52.957191,-67.712730,882.396973,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
16566,52.957191,-67.629402,877.964050,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."


In [25]:
dfpr11TG.TER_GUIDE.value_counts()

6cdefg    2359
6opqr     1609
6ab       1420
5bcd      1231
4bc       1057
6hi       1028
6j         950
6mn        923
3ab        652
5a         552
6kl        507
5ef        401
4f         389
2b         388
3d         367
4de        356
3c         356
5g         347
5hi        342
1a         338
4gh        304
5jk        244
2c         166
2a         143
4a         139
Name: TER_GUIDE, dtype: int64

In [ ]:
TG4a = dfpr11TG[dfpr11TG.TER_GUIDE == "4a"]
TG4a

In [ ]:
TG4a.to_csv("/home/mlopez/EXEC/Tg_csv/TG4a11.csv")

# ---- Create GeoJson file ----

In [ ]:
geometry = dftg11TG["geometry"]

In [ ]:
crs = {'init': "epsg:4326"}
gdf = GeoDataFrame(dftg11TG, crs=crs, geometry=geometry)

In [ ]:
gdf.to_file("TG11tg.json", driver="GeoJSON")